In [91]:
# Проверка работоспособности системы
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import Image
print(tf.__version__)

# Проверка подключения графического процессора
print("Доступные устройства GPU:")
for device in tf.config.experimental.list_physical_devices('CPU'):
    print(device)

os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

2.15.0
Доступные устройства GPU:
PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')


In [92]:
# Создание датасета для обучения

# Функции загрузки и предобработки к каждому элементу датасета
def load_and_preprocess_train_image(image_path, label):
        img = tf.io.read_file(image_path)
        img = tf.image.decode_png(img, channels=1)  # Черно-белые изображения
        img = tf.image.resize(img, [128, 128])
        img = tf.image.convert_image_dtype(img, tf.float32)
        return img, label

# Функция, которая создает датасет из папки с изображениями
def import_train_data(image_dir):
    # Получение списка файлов с изображениями
    image_files = []
    for papka in os.listdir(image_dir):
        papka_dir = image_dir + '/' + papka
        for file in os.listdir(papka_dir):
            if file.endswith('.png'):
                image_files.append(os.path.join(papka_dir, file))

   
    print("Количество изображений:",len(image_files))
   
    # Создание списка меток 
    labels = []
    dict_labels = dict()
    for i in range(0, len(image_files)):
         # Создание индекса графика
        label = tuple(((image_files[i].split('.png')[0]).split("\\")[1]).split('_'))
        labels.append(i)
        dict_labels[label] = i

    print(dict_labels)
    print("Количество меток:",len(set(labels)))
    print("Распределение меток:",labels)

    # Создание датасета с использованием tf.data.Dataset.from_tensor_slices
    dataset = tf.data.Dataset.from_tensor_slices((image_files, labels))
    # Применение функции загрузки и предобработки к каждому элементу датасета
    dataset = dataset.map(load_and_preprocess_train_image)
    
    return dataset, dict_labels, len(dict_labels)

In [93]:
# Путь к папке с изображениями для обучения
train_image_dir = 'dataset_№1/train/'


train_dataset, train_kodificator, output_classes = import_train_data(train_image_dir)

batch_size = 1  # Количество изображений подаваемых нейросети за одну итерацию обучения
train_dataset = train_dataset.batch(batch_size)
print("Кодификатор:", train_kodificator)
print("Длина кодификатора:", len(train_kodificator))
print("Количество классов: ", output_classes)

Количество изображений: 45
{('test1', '0'): 0, ('test1', '1'): 1, ('test1', '10'): 2, ('test1', '2'): 3, ('test1', '3'): 4, ('test1', '4'): 5, ('test1', '5'): 6, ('test1', '6'): 7, ('test1', '7'): 8, ('test1', '8'): 9, ('test1', '9'): 10, ('test2', '0'): 11, ('test2', '1'): 12, ('test2', '10'): 13, ('test2', '2'): 14, ('test2', '3'): 15, ('test2', '4'): 16, ('test2', '5'): 17, ('test2', '6'): 18, ('test2', '7'): 19, ('test2', '8'): 20, ('test2', '9'): 21, ('test3', '0'): 22, ('test4', '0'): 23, ('test4', '1'): 24, ('test4', '10'): 25, ('test4', '2'): 26, ('test4', '3'): 27, ('test4', '4'): 28, ('test4', '5'): 29, ('test4', '6'): 30, ('test4', '7'): 31, ('test4', '8'): 32, ('test4', '9'): 33, ('test5', '0'): 34, ('test5', '1'): 35, ('test5', '10'): 36, ('test5', '2'): 37, ('test5', '3'): 38, ('test5', '4'): 39, ('test5', '5'): 40, ('test5', '6'): 41, ('test5', '7'): 42, ('test5', '8'): 43, ('test5', '9'): 44}
Количество меток: 45
Распределение меток: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 1

In [97]:
# Создание модели
model = Sequential()

# Первый сверточный слой
model.add(Conv2D(32, (4, 4), activation='relu', input_shape=(128, 128, 1)))

# Пулинг для уменьшения размера
model.add(MaxPooling2D(pool_size=(4, 4)))

# Второй сверточный слой
model.add(Conv2D(64, (2, 2), activation='relu'))

# Пулинг для уменьшения размера
model.add(MaxPooling2D(pool_size=(2, 2)))

# Спец. слой, который переводит матрицу в вектор
model.add(Flatten())

# Полносвязный слой
model.add(Dense(150, activation='relu'))

# Выходной слой с нейронами (по одному для каждого класса) и softmax активацией
model.add(Dense(output_classes, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 125, 125, 32)      544       
                                                                 
 max_pooling2d_16 (MaxPooli  (None, 31, 31, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_17 (Conv2D)          (None, 30, 30, 64)        8256      
                                                                 
 max_pooling2d_17 (MaxPooli  (None, 15, 15, 64)        0         
 ng2D)                                                           
                                                                 
 flatten_8 (Flatten)         (None, 14400)             0         
                                                                 
 dense_16 (Dense)            (None, 150)              

In [104]:
history = model.fit(train_dataset, epochs=1, validation_data=train_dataset)

45/45 [==============================] - 2s 52ms/step - loss: 0.0600 - accuracy: 0.9778 - val_loss: 0.0011 - val_accuracy: 1.0000


In [105]:
test_loss, test_accuracy = model.evaluate(train_dataset)
print ('Точность нейросети:', test_accuracy*100)

45/45 [==============================] - 0s 6ms/step - loss: 0.0011 - accuracy: 1.0000
Точность нейросети: 100.0


In [120]:
def load_and_preprocess_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_png(img, channels=1)  # Черно-белые изображения
    img = tf.image.resize(img, [128, 128])
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.expand_dims(img, axis=0)  # Создание батча из одного изображения
    return img

# Пример пути к новому изображению
img_path = 'dataset_№1/train/test5/test5_0.png'
img = load_and_preprocess_image(img_path)


# Получение предсказания
predictions = model.predict(img)

# Определение класса с наибольшей вероятностью
predicted_class = tf.argmax(predictions, axis=1).numpy()[0]
print('Predicted class:', predicted_class)

1/1 [==============================] - 0s 49ms/step
Predicted class: 34


In [123]:
# ГЕНЕРАЦИЯ ИЗОБРАЖЕНИЯ ПО МЕТКЕ

def generate_image_for_label(model, label, input_shape=(128, 128, 1), steps=10000, learning_rate=1.0):
    # Инициализация случайного шума
    input_img = tf.Variable(tf.random.uniform(input_shape))
    
    # Оптимизатор
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    
    for step in range(steps):
        with tf.GradientTape() as tape:
            tape.watch(input_img)
            predictions = model(tf.expand_dims(input_img, axis=0))
            loss = -tf.reduce_mean(predictions[:, label])
        
        grads = tape.gradient(loss, input_img)
        optimizer.apply_gradients([(grads, input_img)])
        
        # Нормализация изображения для предотвращения чрезмерных значений
        input_img.assign(tf.clip_by_value(input_img, 0.0, 1.0))
        
        if step % 100 == 0:
            print(f'Step {step}, Loss: {loss.numpy()}')
    
    return input_img.numpy()

# Пример метки
label = 0  # Замените на вашу метку

# Генерация изображения
generated_img = generate_image_for_label(model, label)

# Визуализация изображения
plt.imshow(generated_img.squeeze(), cmap='gray')
plt.title(f'Generated Image for Label {label}')
plt.show()


Step 0, Loss: -0.027414830401539803


KeyboardInterrupt: 

In [107]:
#model.save("МОДЕЛИ\model_dataset_1.h5")

c:\Users\gerce\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
